In [17]:
import pandas as pd
import joblib
import pymysql
from sklearn.ensemble import RandomForestClassifier

In [18]:
model = joblib.load('rf_fraud_model.pkl')
encoders = joblib.load('rf_encoders.pkl')

In [19]:
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='Root',
    database='dbms'
)

cursor = conn.cursor()

In [20]:
cursor.execute("SELECT txn_id, amount, recipient, location, device_type, txn_type, frequency FROM transactions WHERE is_fraud IS NULL")
rows = cursor.fetchall()

In [21]:
columns = ['txn_id', 'amount', 'recipient', 'location', 'device_type', 'txn_type', 'frequency']
df = pd.DataFrame(rows, columns=columns)

In [22]:
def encode_with_fallback(value, encoder, fallback=-1):
    try:
        return encoder.transform([value])[0]
    except:
        return fallback

for col in ['recipient', 'location', 'device_type', 'txn_type']:
    df[col] = df[col].apply(lambda x: encode_with_fallback(x, encoders[col]))

In [24]:
if not df.empty:
    for col in ['recipient', 'location', 'device_type', 'txn_type']:
        df[col] = df[col].apply(lambda x: encode_with_fallback(x, encoders[col]))

    X = df[['amount', 'recipient', 'location', 'device_type', 'txn_type', 'frequency']]
    predictions = model.predict(X)

    for i, row_id in enumerate(df['txn_id']):
        cursor.execute("UPDATE transactions SET is_fraud = %s WHERE txn_id = %s", (int(predictions[i]), row_id))

    conn.commit()
    print("✅ Predictions done and updated in the database.")
else:
    print("⚠️ No new transactions found for prediction.")


✅ Predictions done and updated in the database.
